In [19]:
import pandas as pd
import numpy as np
import string
import re
import os
import glob
import ftfy
import ftfy.bad_codecs
import emoji
import langid
from langid.langid import LanguageIdentifier, model
import spacy
from spacy.attrs import LOWER, POS, ENT_TYPE, IS_ALPHA, TAG
from spacy.tokenizer import Tokenizer
from spacy.lang.en import English

In [22]:
df_cleaned = pd.read_json("cleaning_test_output.json", orient = 'records', lines = True) 

In [88]:
%time
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType, ArrayType
import spacy

def emoji_remover(doc):
    def keep_token(token):
        if not any(j in token.text for j in emoji.UNICODE_EMOJI):
            return token
    emoji_removed = list(filter(keep_token,doc))
    result = ' '.join(token.string for token in emoji_removed)
    return tokenizer(result)

def remove_tokens_onmatch(doc):
    def keep_token(token):
        if (token.pos_ not in ('PUNCT','SYM','SPACE')) & (token.tag_ not in {"URL","EMAIL"}):
            return token
    removed_result = list(filter(keep_token,doc))
    result = ''.join(token.string for token in removed_result)
    return tokenizer(result)

nlp = spacy.load('en')
nlp.add_pipe(emoji_remover, name='emoji_remover', first=True)
nlp.add_pipe(remove_tokens_onmatch, name='remove_tokens_onmatch', first=False)
tokenizer = English().Defaults.create_tokenizer(nlp)
print(nlp.pipe_names)

CPU times: user 8 µs, sys: 1 µs, total: 9 µs
Wall time: 19.1 µs
['emoji_remover', 'tagger', 'parser', 'ner', 'remove_tokens_onmatch']


In [77]:
test = df_cleaned.body.values.tolist()

In [92]:
%time
df_words = list(map(lambda x : nlp(u"{}".format(x)), test))

CPU times: user 6 µs, sys: 1e+03 ns, total: 7 µs
Wall time: 37.9 µs


In [97]:
df_cleaned['semi_cleaned'] = np.array(df_words).reshape((len(df_words),1))

In [ ]:
import string 
  
# initializing string   
test_string = ""
  
# using sum() + strip() + split() 
# to count words in string 
res = sum([i.strip(string.punctuation).isalpha() for i in test_string.split()]) 
  
# printing result 
print ("The number of words in string are : " +  str(res)) 

In [ ]:
def doc_length (df_str):
    